The following notebook adapts some code from the TensorFlow tutorial: https://www.tensorflow.org/tutorials/keras/classification

In [1]:
import keras
from keras import Sequential
from keras.layers  import Dense, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Helper libraries
import numpy as np  # Keras needs an older version on numpy
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from RProp import RProp
from Wame import Wame

## Import Landsat

In [3]:
def extract_data_lables(file):
    data, lables = [], []
    with open(file, "r") as f:
        for row in f:
            row = row.split()
            row = [int(i) for i in row]
            lables.append(row.pop(-1))
            data.append(row)
        return np.array(data), np.array(lables)

In [4]:
train_data, train_lables = extract_data_lables("data\landsat\sat.trn")

In [5]:
test_data, test_lables  = extract_data_lables("data\landsat\sat.tst")

In [6]:
print(len(train_data))
print(len(test_data))

4435
2000


### Clean Lables

In [7]:
train_lables -= 1
test_lables -= 1

In [8]:
class_names = {
    0:'red soil'
    ,1:'cotton crop'
    ,2:'grey soil'
    ,3:'damp grey soil'
    ,4:'soil with vegetation stubble'
    ,5:'mixture class (all types present)'
    ,6:'very damp grey soil'
}

## Preprocess Images

In [9]:
# scale pixels to 0-1
train_data = train_data / 255.0
test_data = test_data / 255.0

TODO: Find a way to optimise these layers (will be needed for report)

## One-hot encode lables
Class lables need to be in an nxc array format

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
ohe = OneHotEncoder(sparse=False)
train_y = ohe.fit_transform(train_lables.reshape(-1, 1))
test_y = ohe.transform(test_lables.reshape(-1, 1))

In [12]:
train_data.shape

(4435, 36)

In [13]:
train_y.shape

(4435, 6)

## Functions for creating and testing models

In [34]:
def train_test_model(model, optimiser='adam'):
    """Trains a model using 25% validation, then tests the model with the best validation performance"""
    
    model.compile(optimizer=optimiser
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy']
             )
    
    history =  model.fit(
        train_data
        , train_y
        , epochs=50
        , validation_split=0.25
        , verbose=0
        ,callbacks = [
            EarlyStopping(verbose=False, patience=20, monitor='val_loss'),
            ModelCheckpoint('TestModel-progress', monitor='val_loss', verbose=False, save_best_only=True)],
    )
    
    test_loss, test_acc = model.evaluate(test_data,  test_y, verbose = 0)
    return (test_loss, test_acc)   

In [29]:
def create_seq_model(n_hidden=1, n_nodes=36):
    model = Sequential()
    # First hidden layer
    model.add(Dense(n_nodes, input_dim=36))  # input_dim easier to use than input_shape
    model.add(Activation('relu'))
    # Additional hidden layers
    for i in range(n_hidden-1):
        model.add(Dense(n_nodes))
        model.add(Activation('relu'))
    # output layer
    model.add(Dense(6))  
    model.add(Activation('sigmoid'))
    return model

In [30]:
def train_test_grid(n_hidden_lst, n_nodes_lst, optimiser):
    test_losses = np.zeros((len(n_hidden_lst), len(n_nodes_lst)))
    test_accs = np.zeros((len(n_hidden_lst), len(n_nodes_lst)))

    for i, n_hidden in enumerate(n_hidden_lst):
        for j, n_nodes in enumerate(n_nodes_lst):
            model = create_seq_model(n_hidden, n_nodes)
            test_result = train_test_model(model, optimiser)
            test_losses[i, j] = test_result[0]
            test_accs[i, j] = test_result[1]
    
    return test_accs, test_losses

#### Initial model check

In [17]:
from keras.callbacks import LambdaCallback

In [32]:
model = create_seq_model(1, 36)

#print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()))

model.compile(optimizer=Wame(clipnorm=1.0)
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy']
             )
model.fit(
        train_data
        , train_y
        , epochs=100
        , validation_split=0.25
        , verbose=0
 #       , callbacks = [print_weights]
    )
    
test_loss, test_acc = model.evaluate(test_data,  test_y, verbose = 0)

print(test_acc)

0.806


In [33]:
model.predict(train_data)

array([[3.6206841e-04, 2.6822090e-07, 2.7121603e-02, 6.7287683e-04,
        8.9406967e-08, 1.2874603e-05],
       [1.6713142e-04, 6.8545341e-07, 7.8963637e-03, 1.0628700e-03,
        3.8743019e-07, 9.5754862e-05],
       [8.5324049e-05, 1.0430813e-06, 4.0685833e-03, 1.4330447e-03,
        1.4603138e-06, 2.7629733e-04],
       ...,
       [6.7105889e-04, 2.9325485e-05, 1.2126565e-04, 1.6492605e-04,
        7.3313713e-06, 2.2798777e-05],
       [4.5374036e-04, 1.5437603e-05, 2.1111965e-04, 2.6634336e-04,
        6.8843365e-06, 4.8696995e-05],
       [3.9741397e-04, 1.1622906e-05, 2.5987625e-04, 3.2609701e-04,
        6.8631471e-06, 7.0584261e-05]], dtype=float32)

#### Test various 'adam' configurations

In [20]:
n_hidden_lst = [1, 2, 3]
n_nodes_lst = [10, 20, 30, 40]
optimiser = 'adam'

In [27]:
test_accs_adam, test_losses_adam = train_test_grid(n_hidden_lst, n_nodes_lst, optimiser)
print(test_accs_adam)

[[0.797  0.8005 0.8085 0.811 ]
 [0.789  0.8065 0.8075 0.793 ]
 [0.8075 0.8135 0.805  0.779 ]]


##### Test various 'wame' configurations

In [35]:
test_accs_wame, test_losses_wame = train_test_grid(n_hidden_lst, n_nodes_lst, Wame(clipnorm=1.0))
print(test_accs_wame)

[[0.5915 0.6935 0.7965 0.777 ]
 [0.785  0.4075 0.8025 0.7965]
 [0.235  0.3315 0.2305 0.341 ]]


In [23]:
# train_test_model(model)

In [24]:
# model = create_seq_model(n_hidden=2)
# train_test_model(model)

### Train

In [25]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# plt.show()